In [1]:
%env DATA_DIR=$HOME/datasets
%env EXPERIMENT_BASE=$HOME/experiments/ood_flows
%env LOG_LEVEL=INFO
%env BATCH_SIZE=64
%env OPTIM_LR=0.001
%env OPTIM_M=0.8
%env TRAIN_EPOCHS=100
%env EXC_RESUME=1
%env DATASET_NAME=QPM_species
%env MANIFOLD_D=512
%env MODEL_NAME=resnet

env: DATA_DIR=$HOME/datasets
env: EXPERIMENT_BASE=$HOME/experiments/ood_flows
env: LOG_LEVEL=INFO
env: BATCH_SIZE=64
env: OPTIM_LR=0.001
env: OPTIM_M=0.8
env: TRAIN_EPOCHS=100
env: EXC_RESUME=1
env: DATASET_NAME=QPM_species
env: MANIFOLD_D=512
env: MODEL_NAME=resnet


In [2]:
import lightning.pytorch as pl
import numpy as np
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers.wandb import WandbLogger

from config import Config, load_config
from datasets import get_data
from models import get_model

ModuleNotFoundError: No module named 'lightning'

In [ ]:
# initialize the RNG deterministically
np.random.seed(42)
torch.manual_seed(42)

config = load_config()

# initialize data attributes and loaders
get_data(config)
config.print_labels()

assert config.datamodule
wandb_logger = WandbLogger(project="uq_project", log_model="all")

model = get_model(config)
wandb_logger.watch(model, log="all")

checkpoint_callback = ModelCheckpoint(monitor="val_accuracy", mode="max")
trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=config.train_epochs,
    callbacks=[checkpoint_callback],
    reload_dataloaders_every_n_epochs=5,
)
trainer.fit(model=model, datamodule=config.datamodule)